2019/2020 - MDS-1 - Extracción, Transformación y Carga - Etl

Raquel Fort Serra

# Examen ETL: SPARK 12+1/02

Se podrá utilizar toda la información que se encuentra en el campus. 

Se va a trabajar sobre varios ficheros de datos:

Usuarios: id_usuario::sexo::edad::id_profesion::codigo_postal

Peliculas: id_pelicula::titulo (año)::tipo1|tipo2|tipo3

Ratings: id_pelicula::id_usuario::puntuacion::fecha_timestamp

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

## Primera tarea: Inicializar spark context y cargar los datos desde los ficheros.

In [1]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")

In [2]:
usuar = "users.dat"
usuar = sc.textFile(usuar)

In [3]:
peli = "movies.dat.txt"
peli = sc.textFile(peli)

In [4]:
ratings = "ratings.dat"
ratings = sc.textFile(ratings)

In [5]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

Parseamos los datos, determinamos cómo se separan las columnas (::) y convertimos la fecha a año ya que está en formato timestamp

In [6]:
from datetime import datetime
usuar = usuar.map(lambda x: x.split("::"))
peli = peli.map(lambda x: x.split("::"))
ratings = ratings.map(lambda x: x.split("::")).map(lambda x: [x[0], x[1], int(x[2]), datetime.fromtimestamp(float(x[3])).year])

In [7]:
usuar.take(1)

[['1', 'F', '1', '10', '48067']]

In [8]:
peli.take(1)

[['1', 'Toy Story (1995)', "Animation|Children's|Comedy"]]

In [9]:
ratings.take(1)

[['1', '1193', 5, 2000]]

## Segunda tarea: Media de puntuaciones globales por año. ¿Hay algún año significativamente distinto?

Únicamente vamos a utilizar la base de datos de ratings, ya que contiene las columnas que vamos a necesitar: puntuación y fecha. 
En primer lugar agrupamos por año todas las puntuaciones:

In [10]:
totalrating = ratings.map(lambda x: (x[3], int(x[2]))) 
totalrating = totalrating.reduceByKey(lambda x, y: x + y)

In [11]:
totalrating.take(1)

[(2000, 3248432)]

Luego contamos cuántas veces se ha votado por año para calcular la media

In [12]:
numrat_año = ratings.map(lambda x: (x[3], 1))
numrat_año = numrat_año.reduceByKey(lambda x, y: x + y)

In [13]:
numrat_año.take(1)

[(2000, 904757)]

Ahora juntamos las dos bases de datos (totalrating y numrat_año) para tener para cada año, las puntuaciones totales y el número de veces que se ha votado

In [14]:
año = totalrating.join(numrat_año)

In [15]:
año.take(3)

[(2000, (3248432, 904757)), (2001, (239037, 68058)), (2002, (83171, 24046))]

Y calculamos la media

In [16]:
media = año.map(lambda x: (x[0], x[1][0]/x[1][1]))

In [17]:
media.take(3)

[(2000, 3.5903916742285498),
 (2001, 3.5122542537247643),
 (2002, 3.458828911253431)]

In [18]:
#redondeo porque tenía demasiados decimales
media = media.map(lambda x: (x[0], round(x[1],2)))

In [19]:
#Y aquí tenemos las medias para cada año
media.take(4)

[(2000, 3.59), (2001, 3.51), (2002, 3.46), (2003, 3.49)]

Como podemos ver, estas son las medias para cada año:
- 2000 = 3.59
- 2001 = 3.51
- 2002 = 3.46
- 2003 = 3.49

Todas las medias son muy parecidas, no hay ningun año significativamente distinto

## Tercera pregunta: ¿Cuál es la película más votada por los mayores de 60? 

En primer lugar, nos quedamos con el id. usuario para los de la edad >60, ya que es la variable en común con la base de datos que contiene la puntuación. 

In [20]:
#nos quedamos con las variables que necesitamos
usuarios = usuar.map(lambda x: (x[0], int(x[2])))

In [21]:
usuarios.take(3)

[('1', 1), ('2', 56), ('3', 25)]

In [22]:
#filtramos las observaciones de los usuarios mayores a 60 años
usuarios60 = usuarios.filter(lambda x: x[1] >= 60)

In [23]:
usuarios60.take(3) #no hay observaciones¿?

[]

In [24]:
usuarios.max(lambda x: x[1]) #vale, la edad máxima de los usuarios es 56 años

('2', 56)

No hay usuarios mayores de 60 años, la edad máxima de personas que han votado es de 56 años, así que no podemos responder a la pregunta

## Cuarta pregunta: ¿Cuál es la puntuación media de las peliculas de acción del año 2000?

En primer lugar miro en el tipo de película cómo está escrito que una película es de acción

In [28]:
peli.map(lambda x: x[2]).distinct().collect() #veo que es 'Action'

['Comedy|Romance|Thriller',
 'Drama|Sci-Fi',
 "Animation|Children's|Musical|Romance",
 'Horror',
 'Adventure|Romance',
 'Comedy|Romance',
 "Adventure|Animation|Children's|Sci-Fi",
 'Adventure|Comedy',
 "Animation|Children's|Comedy",
 'Action|Sci-Fi|War',
 'Comedy|Fantasy|Romance|Sci-Fi',
 "Adventure|Children's|Drama",
 'Romance',
 'Adventure|Drama|Romance|Sci-Fi',
 'Action|Drama|Romance|Thriller',
 'Action|Sci-Fi|Western',
 'Drama|Film-Noir|Thriller',
 'Adventure|Drama',
 'Crime|Film-Noir|Mystery|Thriller',
 'Animation|Comedy',
 'Adventure',
 'Comedy|Crime',
 "Animation|Children's|Musical",
 'Action|Adventure|Crime|Drama',
 "Children's",
 'Action|Adventure|Drama|Thriller',
 'Crime|Horror|Mystery|Thriller',
 'Horror|Sci-Fi|Thriller',
 'Adventure|Comedy|Romance',
 'Romance|Thriller',
 'Action|Adventure|Crime',
 'Action|Adventure|Horror|Thriller',
 'Crime|Thriller',
 "Children's|Drama",
 'Comedy|Fantasy|Romance',
 "Action|Adventure|Animation|Children's|Fantasy",
 'Action|Adventure|Animati

Primero tenemos que seleccionar las películas que tengan como tipo "Action", junto con su id de película, para luego (a partir del id de película) buscar la puntuacion de cada una y sacar la media.  

In [29]:
#seleccionamos las peliculas de acción
peliAction = peli.filter(lambda x: 'Action' in x[2]) 
peliAction.take(1)

[['6', 'Heat (1995)', 'Action|Crime|Thriller']]

In [30]:
#no nos quedamos con los títulos porque no nos interesa
peliAction = peliAction.map(lambda x: (x[0], x[2]))
peliAction.take(1)

[('6', 'Action|Crime|Thriller')]

Ahora seleccionamos de la base de datos de ratings, los id de pelicula con la puntuacion para los que sean del año 2000

In [31]:
#Seleccionamos las variables de la base de datos de ratings que nos interesa, y transformmos a 
#numérica la puntuación
ratAction = ratings.map(lambda x: (x[0], int(x[2]), x[3]))

In [32]:
ratAction.take(1)

[('1', 5, 2000)]

In [33]:
#filtramos las observaciones para el año 2000
ratAction2 = ratAction.filter(lambda x: x[2] == 2000)
ratAction2.take(1)

[('1', 5, 2000)]

In [34]:
ratAction2.take(10)

[('1', 5, 2000),
 ('1', 3, 2000),
 ('1', 3, 2000),
 ('1', 4, 2000),
 ('1', 3, 2000),
 ('1', 5, 2000),
 ('1', 5, 2000),
 ('1', 4, 2000),
 ('1', 4, 2000),
 ('1', 4, 2000)]

Nos interesa la media de todas las películas de Acción, no nos interesa el id de película, así que utilizamos el mismo índice para todas ('Acción') y sumamos todas las puntuaciones para luego calcular la media 

In [44]:
ratActionTotal = ratAction2.map(lambda x: ('Acción', x[1]))

In [45]:
#aquí tenemos la suma de todas las puntuaciones de las películas de acción
ratActionTotal = ratActionTotal.reduceByKey(lambda x, y: x + y) 

In [46]:
ratActionTotal.take(1) 

[('Acción', 3248432)]

Calculamos el número de veces que se ha votado para la obtención de la media

In [41]:
mediaAction = ratAction2.map(lambda x: ('Acción', 1))
mediaAction = mediaAction.reduceByKey(lambda x, y: x + y)

In [42]:
mediaAction.take(1)

[('Acción', 904757)]

In [47]:
mediaAction2 = ratActionTotal.join(mediaAction)
mediaAction2.take(1)

[('Acción', (3248432, 904757))]

In [53]:
mediaAction3 = mediaAction2.map(lambda x: (x[0], round((x[1][0]/x[1][1]),2)))
mediaAction3.take(1)                             

[('Acción', 3.59)]

## Quinta pregunta: ¿ Cuál es el año en que mayor número de usuarios votaron?

Primero guardamos lo que nos interesa de ratings: el año, y el id de usuario

In [54]:
ratusuarios = ratings.map(lambda x: (x[1], x[3])).distinct()

In [55]:
ratusuarios.take(3)

[('2435', 2000), ('3524', 2002), ('2306', 2001)]

Ahora contamos cuantos hay por año 

In [56]:
numYear = ratusuarios.map(lambda x: (x[1], 1))
numYear = numYear.reduceByKey(lambda x, y: x + y)

In [57]:
numYear.take(5)

[(2000, 3678), (2001, 3289), (2002, 2971), (2003, 1601)]

In [58]:
numYear.max(lambda x: x[1])

(2000, 3678)

En el año 2000 votaron mayor número de usuarios (3678 usuarios)

## Sexta pregunta: ¿ Cuál es la película con mejor puntación media?

Primero nos quedamos con las columnas que nos interesan en cada base de datos:  con el id de pelicula y el título (para la base de datos de 'peli') y con el id de película y la puntuación (para la base de datos de ratings).

In [59]:
peliculas = peli.map(lambda x: (x[0], x[1]))

In [60]:
peliculas.take(2)

[('1', 'Toy Story (1995)'), ('2', 'Jumanji (1995)')]

In [61]:
puntu = ratings.map(lambda x: (x[0], x[2]))
puntu.take(2)

[('1', 5), ('1', 3)]

Sumamos las puntuaciones para cada película 

In [62]:
puntupeli = puntu.map(lambda x: (x[0], x[1]))
puntupeli = puntupeli.reduceByKey(lambda x, y: x + y)
puntupeli.take(2)

[('4334', 564), ('4986', 118)]

Vemos cuantas veces se ha votado para cada pelicula

In [63]:
numpeli = puntu.map(lambda x: (x[0],1))
numpeli = numpeli.reduceByKey(lambda x, y: x + y)
numpeli.take(2)

[('4334', 157), ('4986', 27)]

Unimos las dos bases de datos

In [64]:
puntotal = puntupeli.join(numpeli)
puntotal.take(3)

[('4334', (564, 157)), ('4986', (118, 27)), ('2265', (845, 232))]

Y calculamos las medias, redondeando con dos decimales.

In [65]:
mediaPeli = puntotal.map(lambda x: (x[0], round((x[1][0]/x[1][1]),2)))
mediaPeli.take(3)

[('4334', 3.59), ('4986', 4.37), ('2265', 3.64)]

In [66]:
mediaPeliFinal = mediaPeli.join(peliculas)
mediaPeliFinal.take(2)

[('2250', (3.07, "Men Don't Leave (1990)")),
 ('2265', (3.64, 'Nothing But Trouble (1991)'))]

In [67]:
mediaPeliFinal.max(lambda x: x[1])

('283', (4.96, 'New Jersey Drive (1995)'))

Obtenemos que la película con la nota media más alta es New Jersey Drive (1995), con un 4.96

In [68]:
sc.stop()